In [1]:
if (!require(forecast)){
    installed <- T
    install.packages("forecast")
}
if (!require(dplyr)){
    install.packages("dplyr")
    require(dplyr)
}

Loading required package: forecast
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
ericsson_prediction <- function(defect_backlog_path, weeks_to_predict_path, forecast_summary_path){

defect_backlog = read.csv(file=defect_backlog_path, header=TRUE, sep=",")
weeks_to_predict = read.csv(file=weeks_to_predict_path, header=TRUE, sep=",")
    
weekly_inflow_ts = ts(defect_backlog$inflow_all)
numbers_of_weeks_to_predict = weeks_to_predict$number

forecasts <- c()
actuals <- c()
errors <- c()
method <- "ericsson"
    
for(i in 1:nrow(weeks_to_predict)){
    db_1 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-1) %>% select (backlog_all)
    di_1 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-1) %>% select (inflow_all)
    di_2 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-2) %>% select (inflow_all)
    di_3 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-3) %>% select (inflow_all)
    do_1 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-1) %>% select (outflow_all)
    do_2 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-2) %>% select (outflow_all)
    do_3 <- defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]-3) %>% select (outflow_all)
   
    di <- (di_1 + di_2 + di_3)/3
    do <- (do_1 + do_2 + do_3)/3
    db <- db_1 + di - do

    forecasts[i] = round(db)
    actuals[i] = defect_backlog %>% filter (number == numbers_of_weeks_to_predict[i]) %>% select (backlog_all)
    ae <- as.integer(forecasts[i]) - as.integer(actuals[i])
    errors[i] = abs(ae)
}

result <- cbind(Method = method, Week=numbers_of_weeks_to_predict, Actual=actuals, Forecast=forecasts, Absolute_Error = errors)
write.table(result, file = forecast_summary_path, sep=",")
}

In [8]:
ericsson_prediction("/Users/paulina/magisterka/eclipse/converted_data/platform-backlog_30_12_12-4_1_16.csv",
                 "/Users/paulina/magisterka/eclipse/samples/samples_30_12_12-4_1_16.csv",
                 "/Users/paulina/magisterka/eclipse/forecasts/ericsson_30_12_12-4_1_16.csv")

In [9]:
read.csv(file="/Users/paulina/magisterka/eclipse/forecasts/ericsson_30_12_12-4_1_16.csv", header=TRUE, sep=",")

Method,Week,Actual,Forecast,Absolute_Error
ericsson,16,625,614,11
ericsson,26,721,722,1
ericsson,27,738,748,10
ericsson,33,878,887,9
ericsson,44,983,971,12
ericsson,51,1092,1093,1
ericsson,62,1250,1213,37
ericsson,65,1309,1324,15
ericsson,70,1285,1270,15
ericsson,83,1380,1393,13
